In [37]:
from src.util.io.io_util import IOUtil
from src.util.path.local_path import LocalPath
from src.util.mysql.dataframe_and_mysql import DataframeAndMysql

In [38]:
local_path = LocalPath()
file_path = local_path.get_key_folder() + "/prefect_trip_key.txt"

io_util = IOUtil()
database_info = io_util.read_key_file_data(file_path)

In [39]:
dataframe_and_mysql = DataframeAndMysql(database_name="perfect_trip_db", 
                                        username=database_info["username"], 
                                        password=database_info["password"], 
                                        hostname=database_info["hostname"], 
                                        port=database_info["port"])

In [40]:
data = dataframe_and_mysql.dataframe_from_mysql("user_master")
data

,user_id,username,password,first_name,last_name,nickname,tax_id,gender,user_group,phone_number,country,avatar,action,change_id,created_date,last_modified_date
0,2,test,e10adc3949ba59abbe56e057f20f883e,None,None,None,None,None,None,None,None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,None,0,2024-07-28 11:58:22,2024-07-28 11:58:22
1,3,test1,e10adc3949ba59abbe56e057f20f883e,None,None,None,None,None,None,None,None,None,None,0,2024-07-28 12:00:42,2024-07-28 12:00:42


In [41]:
binary_image = data[data["user_id"] == 2]["avatar"]
binary_image

0    b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
Name: avatar, dtype: object

In [47]:
import pymysql
from PIL import Image
import io

conn = pymysql.connect(host=database_info["hostname"], 
                       user=database_info["username"], 
                       password=database_info["password"], 
                       db="perfect_trip_db", 
                       charset="utf8mb4", 
                       port=8995)

try:
    with conn.cursor() as cursor:
        sql = "SELECT avatar FROM user_master WHERE user_id = 2;"
        cursor.execute(sql)

        result = cursor.fetchone()
        if result:
            image_blob = result[0]
        else:
            image_blob = None

finally:
    conn.close()

if image_blob:
    image_stream = io.BytesIO(image_blob)
    image = Image.open(image_stream)
    image.show()
else:
    print("No image found.")